# Feature Selection on Latest LFS Release (2024-07JUL)

**Purpose:** Identify which demographic variables have meaningful association with employment status (PUFNEWEMPSTAT), so we know which variables are worth carefully harmonizing across 105 survey files.

**Setup:**
- Target: PUFNEWEMPSTAT (1=Employed, 2=Unemployed, 3=NILF)
- Population: Working-age (15+)
- Approach: Time-series forecasting on aggregated data
- Only demographic/structural variables are valid features (no data leakage from employment-detail variables)

## 1. Load and Filter

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# Load latest release
df_raw = pd.read_csv('./raw/2024-07JUL.CSV', low_memory=False)
print(f'Raw: {df_raw.shape[0]:,} rows, {df_raw.shape[1]} columns')

# Convert target and age to numeric
df_raw['PUFNEWEMPSTAT'] = pd.to_numeric(df_raw['PUFNEWEMPSTAT'], errors='coerce')
df_raw['PUFC05_AGE'] = pd.to_numeric(df_raw['PUFC05_AGE'], errors='coerce')
df_raw['PUFPWGTPRV'] = pd.to_numeric(df_raw['PUFPWGTPRV'], errors='coerce')

# Filter: working-age (15+) with valid employment status
df = df_raw[(df_raw['PUFC05_AGE'] >= 15) & (df_raw['PUFNEWEMPSTAT'].isin([1, 2, 3]))].copy()
print(f'After filter (age>=15, valid EMPSTAT): {len(df):,} rows')

# Label the target
empstat_labels = {1: 'Employed', 2: 'Unemployed', 3: 'NILF'}
df['empstat_label'] = df['PUFNEWEMPSTAT'].map(empstat_labels)

# Weighted class distribution
w = df['PUFPWGTPRV'].fillna(0)
total_w = w.sum()
print(f'\nWeighted employment status distribution:')
for val, label in empstat_labels.items():
    mask = df['PUFNEWEMPSTAT'] == val
    pct = w[mask].sum() / total_w * 100
    print(f'  {label}: {pct:.1f}% (n={mask.sum():,})')

ModuleNotFoundError: No module named 'scipy'

## 2. Prepare Candidate Features

Only demographic/structural variables that exist BEFORE employment status is determined.

**Why only demographics?** For time-series forecasting, we aggregate person-level data into period-level summaries (e.g., % female, mean age, % with bachelor's degree). Employment-detail variables (occupation, hours, pay) are only filled for employed persons -- using them creates circular logic (data leakage).

In [ ]:
# --- Prepare candidate features ---

# SEX: already clean (1=Male, 2=Female)
df['PUFC04_SEX'] = pd.to_numeric(df['PUFC04_SEX'], errors='coerce')

# AGE: already numeric

# AGE GROUPS (for aggregation in time-series)
df['age_group'] = pd.cut(df['PUFC05_AGE'], 
                         bins=[14, 24, 34, 44, 54, 64, 200],
                         labels=['15-24', '25-34', '35-44', '45-54', '55-64', '65+'])

# MSTAT: passthrough (convert to numeric)
df['PUFC06_MSTAT'] = pd.to_numeric(df['PUFC06_MSTAT'], errors='coerce')

# GRADE: map raw 5-digit PSCED codes to levels 0-8
def grade_to_psced_level(code):
    """Map raw PSCED 5-digit code to top-level 0-8."""
    try:
        code = int(float(code))
    except (ValueError, TypeError):
        return np.nan
    if code == 0 or code == 1000:
        return 0
    if code == 2000:
        return 1
    if 10000 <= code <= 19999:
        return 1
    if 20000 <= code <= 29999:
        return 2
    if 30000 <= code <= 39999:
        return 3
    if 40000 <= code <= 49999:
        return 4
    if 50000 <= code <= 59999:
        return 5
    if 60000 <= code <= 69999:
        return 6
    if 70000 <= code <= 79999:
        return 7
    if 80000 <= code <= 89999:
        return 8
    return np.nan

df['grade_level'] = df['PUFC07_GRADE'].apply(grade_to_psced_level)

psced_labels = {
    0: 'L0: Early Childhood/None',
    1: 'L1: Primary',
    2: 'L2: Lower Secondary',
    3: 'L3: Upper Secondary',
    4: 'L4: Post-Sec Non-Tertiary',
    5: 'L5: Short-Cycle Tertiary',
    6: 'L6: Bachelor',
    7: 'L7: Master',
    8: 'L8: Doctoral'
}
df['grade_label'] = df['grade_level'].map(psced_labels)

# REL: relationship to HH head
df['PUFC03_REL'] = pd.to_numeric(df['PUFC03_REL'], errors='coerce')
# Collapse to: Head, Spouse, Child, Other
def collapse_rel(code):
    try:
        code = int(float(code))
    except (ValueError, TypeError):
        return np.nan
    if code == 1: return 'Head'
    if code == 2: return 'Spouse'
    if code in (3, 4): return 'Child'
    return 'Other'

df['rel_group'] = df['PUFC03_REL'].apply(collapse_rel)

# CURSCH: currently attending school
df['PUFC08_CURSCH'] = pd.to_numeric(df['PUFC08_CURSCH'], errors='coerce')
df['is_in_school'] = df['PUFC08_CURSCH'].isin([1, 2, 3]).astype(float)  # 1/2/3 = Yes (pub/priv/home)
df.loc[df['PUFC08_CURSCH'].isna(), 'is_in_school'] = np.nan

# GRADTECH: TVET graduate
df['PUFC09_GRADTECH'] = pd.to_numeric(df['PUFC09_GRADTECH'], errors='coerce')

# HHSIZE
df['PUFHHSIZE'] = pd.to_numeric(df['PUFHHSIZE'], errors='coerce')

# Summary
features = ['PUFC04_SEX', 'PUFC05_AGE', 'age_group', 'PUFC06_MSTAT', 
            'grade_level', 'rel_group', 'is_in_school', 'PUFC09_GRADTECH', 'PUFHHSIZE']

print('Candidate features and their coverage:')
print(f'{"Feature":<20s} {"Non-null":>10s} {"Null%":>8s} {"Unique":>8s}')
print('-' * 48)
for f in features:
    nn = df[f].notna().sum()
    null_pct = df[f].isna().mean() * 100
    nu = df[f].nunique()
    print(f'{f:<20s} {nn:>10,d} {null_pct:>7.1f}% {nu:>8d}')

## 3. Weighted Demographic Profiles by Employment Status

For each candidate feature, see how its distribution differs across Employed, Unemployed, and NILF.

In [ ]:
# Helper: weighted crosstab
def weighted_crosstab(df, feature, target='PUFNEWEMPSTAT', weight='PUFPWGTPRV'):
    """Compute weighted % distribution of feature within each target class."""
    valid = df[[feature, target, weight]].dropna()
    results = []
    for cls in sorted(valid[target].unique()):
        subset = valid[valid[target] == cls]
        w = subset[weight]
        total_w = w.sum()
        for val in sorted(valid[feature].unique()):
            mask = subset[feature] == val
            pct = w[mask].sum() / total_w * 100 if total_w > 0 else 0
            results.append({
                'empstat': empstat_labels.get(cls, cls),
                feature: val,
                'weighted_pct': round(pct, 2)
            })
    return pd.DataFrame(results)


# --- SEX ---
print('=' * 60)
print('SEX distribution by employment status (weighted %)')
print('=' * 60)
sex_labels = {1: 'Male', 2: 'Female'}
ct = weighted_crosstab(df, 'PUFC04_SEX')
ct['sex_label'] = ct['PUFC04_SEX'].map(sex_labels)
pivot = ct.pivot(index='sex_label', columns='empstat', values='weighted_pct')
print(pivot.to_string())

# --- AGE ---
print(f'\n{"=" * 60}')
print('AGE statistics by employment status (weighted)')
print('=' * 60)
for cls, label in empstat_labels.items():
    subset = df[df['PUFNEWEMPSTAT'] == cls]
    w = subset['PUFPWGTPRV'].fillna(0)
    age = subset['PUFC05_AGE']
    wmean = np.average(age, weights=w) if w.sum() > 0 else 0
    print(f'  {label:12s}: mean={wmean:.1f}, median={age.median():.0f}, '
          f'min={age.min():.0f}, max={age.max():.0f}, n={len(subset):,}')

# AGE GROUP distribution
print(f'\nAGE GROUP distribution by employment status (weighted %)')
ct_age = weighted_crosstab(df, 'age_group')
pivot_age = ct_age.pivot(index='age_group', columns='empstat', values='weighted_pct')
print(pivot_age.to_string())

# --- MSTAT ---
print(f'\n{"=" * 60}')
print('MARITAL STATUS distribution by employment status (weighted %)')
print('=' * 60)
mstat_labels_2024 = {
    1: 'Single/Never Married', 2: 'Married', 3: 'Common-Law/Live-In',
    4: 'Widowed', 5: 'Divorced', 6: 'Separated', 7: 'Annulled', 8: 'Not Reported'
}
ct_ms = weighted_crosstab(df, 'PUFC06_MSTAT')
ct_ms['mstat_label'] = ct_ms['PUFC06_MSTAT'].map(mstat_labels_2024)
pivot_ms = ct_ms.pivot(index='mstat_label', columns='empstat', values='weighted_pct')
print(pivot_ms.to_string())

# --- GRADE (PSCED Level) ---
print(f'\n{"=" * 60}')
print('EDUCATION LEVEL distribution by employment status (weighted %)')
print('=' * 60)
ct_gr = weighted_crosstab(df, 'grade_level')
ct_gr['grade_label'] = ct_gr['grade_level'].map(psced_labels)
pivot_gr = ct_gr.pivot(index='grade_label', columns='empstat', values='weighted_pct')
print(pivot_gr.to_string())

# --- REL (collapsed) ---
print(f'\n{"=" * 60}')
print('RELATIONSHIP TO HH HEAD by employment status (weighted %)')
print('=' * 60)
ct_rel = weighted_crosstab(df, 'rel_group')
pivot_rel = ct_rel.pivot(index='rel_group', columns='empstat', values='weighted_pct')
print(pivot_rel.to_string())

# --- CURSCH ---
print(f'\n{"=" * 60}')
print('CURRENTLY IN SCHOOL by employment status (weighted %)')
print('=' * 60)
ct_sch = weighted_crosstab(df, 'is_in_school')
ct_sch['label'] = ct_sch['is_in_school'].map({0: 'Not in school', 1: 'In school'})
pivot_sch = ct_sch.pivot(index='label', columns='empstat', values='weighted_pct')
print(pivot_sch.to_string())

# --- HHSIZE ---
print(f'\n{"=" * 60}')
print('HOUSEHOLD SIZE by employment status (weighted)')
print('=' * 60)
for cls, label in empstat_labels.items():
    subset = df[df['PUFNEWEMPSTAT'] == cls]
    w = subset['PUFPWGTPRV'].fillna(0)
    hh = subset['PUFHHSIZE'].dropna()
    if len(hh) > 0 and w[hh.index].sum() > 0:
        wmean = np.average(hh, weights=w[hh.index])
        print(f'  {label:12s}: mean={wmean:.2f}, median={hh.median():.0f}')
    else:
        print(f'  {label:12s}: no data')

## 4. Statistical Tests of Association

For each feature, test whether its distribution differs significantly across the 3 employment status classes.

- **Categorical features**: Chi-square test + Cramer's V (effect size)
- **Continuous features**: Kruskal-Wallis test + Eta-squared (effect size)

In [ ]:
def cramers_v(confusion_matrix):
    """Compute Cramer's V from a contingency table."""
    chi2 = stats.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.values.sum()
    r, k = confusion_matrix.shape
    return np.sqrt(chi2 / (n * (min(r, k) - 1))) if n > 0 and min(r, k) > 1 else 0


def eta_squared(groups):
    """Compute eta-squared (effect size for Kruskal-Wallis)."""
    # Using H-statistic approximation: eta^2 = (H - k + 1) / (n - k)
    all_vals = np.concatenate(groups)
    n = len(all_vals)
    k = len(groups)
    H = stats.kruskal(*groups).statistic
    return (H - k + 1) / (n - k) if n > k else 0


results = []

# --- Categorical features ---
cat_features = [
    ('PUFC04_SEX', 'Sex'),
    ('PUFC06_MSTAT', 'Marital Status'),
    ('grade_level', 'Education Level (PSCED)'),
    ('age_group', 'Age Group'),
    ('rel_group', 'Relationship to HH Head'),
    ('is_in_school', 'Currently in School'),
]

for feat, label in cat_features:
    valid = df[[feat, 'PUFNEWEMPSTAT']].dropna()
    if len(valid) < 10:
        continue
    ct = pd.crosstab(valid[feat], valid['PUFNEWEMPSTAT'])
    chi2, p, dof, _ = stats.chi2_contingency(ct)
    cv = cramers_v(ct)
    results.append({
        'feature': label,
        'variable': feat,
        'test': 'Chi-square',
        'statistic': round(chi2, 1),
        'p_value': p,
        'effect_size': round(cv, 4),
        'effect_metric': "Cramer's V",
        'n': len(valid),
    })

# --- Continuous features ---
cont_features = [
    ('PUFC05_AGE', 'Age'),
    ('PUFHHSIZE', 'Household Size'),
]

for feat, label in cont_features:
    valid = df[[feat, 'PUFNEWEMPSTAT']].dropna()
    if len(valid) < 10:
        continue
    groups = [valid[valid['PUFNEWEMPSTAT'] == c][feat].values for c in [1, 2, 3]]
    groups = [g for g in groups if len(g) > 0]
    if len(groups) < 2:
        continue
    H, p = stats.kruskal(*groups)
    eta2 = eta_squared(groups)
    results.append({
        'feature': label,
        'variable': feat,
        'test': 'Kruskal-Wallis',
        'statistic': round(H, 1),
        'p_value': p,
        'effect_size': round(eta2, 4),
        'effect_metric': 'Eta-squared',
        'n': len(valid),
    })

# --- Results table ---
results_df = pd.DataFrame(results).sort_values('effect_size', ascending=False)
results_df['significant'] = results_df['p_value'] < 0.001

print('FEATURE ASSOCIATION WITH EMPLOYMENT STATUS')
print('Ranked by effect size (higher = stronger association)')
print('=' * 90)
print(f'{"Feature":<28s} {"Test":<16s} {"Stat":>10s} {"p-value":>12s} {"Effect Size":>12s} {"Metric":<12s} {"N":>8s}')
print('-' * 90)
for _, r in results_df.iterrows():
    p_str = f'{r["p_value"]:.2e}' if r['p_value'] < 0.001 else f'{r["p_value"]:.4f}'
    print(f'{r["feature"]:<28s} {r["test"]:<16s} {r["statistic"]:>10.1f} {p_str:>12s} {r["effect_size"]:>12.4f} {r["effect_metric"]:<12s} {r["n"]:>8,d}')

print(f'\nEffect size interpretation (Cramer\'s V): <0.10 = negligible, 0.10-0.30 = small, 0.30-0.50 = medium, >0.50 = large')
print(f'Effect size interpretation (Eta-squared): <0.01 = negligible, 0.01-0.06 = small, 0.06-0.14 = medium, >0.14 = large')

## 5. Visualizations

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Demographic Distributions by Employment Status (2024-07JUL)', fontsize=14, fontweight='bold')

colors = {'Employed': '#2196F3', 'Unemployed': '#FF5722', 'NILF': '#9E9E9E'}

# 1. Age distribution
ax = axes[0, 0]
for cls, label in empstat_labels.items():
    subset = df[df['PUFNEWEMPSTAT'] == cls]['PUFC05_AGE']
    ax.hist(subset, bins=range(15, 100, 5), alpha=0.5, label=label, 
            color=colors[label], density=True)
ax.set_xlabel('Age')
ax.set_ylabel('Density')
ax.set_title('Age Distribution')
ax.legend()

# 2. Sex
ax = axes[0, 1]
sex_data = df.groupby(['PUFNEWEMPSTAT', 'PUFC04_SEX']).size().unstack(fill_value=0)
sex_pct = sex_data.div(sex_data.sum(axis=1), axis=0) * 100
sex_pct.index = [empstat_labels[i] for i in sex_pct.index]
sex_pct.columns = ['Male', 'Female']
sex_pct.plot(kind='bar', ax=ax, color=['#42A5F5', '#EF5350'])
ax.set_title('Sex Composition')
ax.set_ylabel('Percentage')
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
ax.legend()

# 3. Education level
ax = axes[0, 2]
grade_data = df.groupby(['PUFNEWEMPSTAT', 'grade_level']).size().unstack(fill_value=0)
grade_pct = grade_data.div(grade_data.sum(axis=1), axis=0) * 100
grade_pct.index = [empstat_labels[i] for i in grade_pct.index]
grade_pct.columns = [psced_labels.get(c, str(c)) for c in grade_pct.columns]
grade_pct.plot(kind='bar', stacked=True, ax=ax, colormap='viridis')
ax.set_title('Education Level')
ax.set_ylabel('Percentage')
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)

# 4. Age group
ax = axes[1, 0]
age_data = df.groupby(['PUFNEWEMPSTAT', 'age_group']).size().unstack(fill_value=0)
age_pct = age_data.div(age_data.sum(axis=1), axis=0) * 100
age_pct.index = [empstat_labels[i] for i in age_pct.index]
age_pct.plot(kind='bar', ax=ax, colormap='Set2')
ax.set_title('Age Group Composition')
ax.set_ylabel('Percentage')
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
ax.legend(fontsize=8)

# 5. Marital status
ax = axes[1, 1]
ms_data = df.groupby(['PUFNEWEMPSTAT', 'PUFC06_MSTAT']).size().unstack(fill_value=0)
ms_pct = ms_data.div(ms_data.sum(axis=1), axis=0) * 100
ms_pct.index = [empstat_labels[i] for i in ms_pct.index]
ms_pct.columns = [mstat_labels_2024.get(c, str(c)) for c in ms_pct.columns]
ms_pct.plot(kind='bar', stacked=True, ax=ax, colormap='tab10')
ax.set_title('Marital Status')
ax.set_ylabel('Percentage')
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)

# 6. Relationship to HH head
ax = axes[1, 2]
rel_data = df.groupby(['PUFNEWEMPSTAT', 'rel_group']).size().unstack(fill_value=0)
rel_pct = rel_data.div(rel_data.sum(axis=1), axis=0) * 100
rel_pct.index = [empstat_labels[i] for i in rel_pct.index]
rel_pct.plot(kind='bar', ax=ax, colormap='Set1')
ax.set_title('Relationship to HH Head')
ax.set_ylabel('Percentage')
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
ax.legend(fontsize=8)

plt.tight_layout()
plt.savefig('./output_v8/feature_selection_distributions.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved: output_v8/feature_selection_distributions.png')

In [ ]:
# Effect size ranking bar chart
fig, ax = plt.subplots(figsize=(10, 5))
plot_df = results_df.sort_values('effect_size', ascending=True)
bars = ax.barh(plot_df['feature'], plot_df['effect_size'], 
               color=['#2196F3' if m == "Cramer's V" else '#FF9800' for m in plot_df['effect_metric']])
ax.set_xlabel('Effect Size')
ax.set_title('Feature Association with Employment Status (Effect Size Ranking)')
ax.axvline(x=0.10, color='gray', linestyle='--', alpha=0.5, label='Small effect threshold')
ax.axvline(x=0.30, color='gray', linestyle=':', alpha=0.5, label='Medium effect threshold')

# Add value labels
for bar, val in zip(bars, plot_df['effect_size']):
    ax.text(bar.get_width() + 0.005, bar.get_y() + bar.get_height()/2, 
            f'{val:.4f}', va='center', fontsize=9)

ax.legend()
plt.tight_layout()
plt.savefig('./output_v8/feature_selection_effect_sizes.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved: output_v8/feature_selection_effect_sizes.png')

## 6. Multicollinearity Check

Check if candidate features are redundant with each other.

In [ ]:
# Compute Cramer's V between all pairs of categorical features
cat_cols = ['PUFC04_SEX', 'age_group', 'PUFC06_MSTAT', 'grade_level', 'rel_group', 'is_in_school']
n_cats = len(cat_cols)
corr_matrix = pd.DataFrame(np.zeros((n_cats, n_cats)), index=cat_cols, columns=cat_cols)

for i in range(n_cats):
    for j in range(i, n_cats):
        if i == j:
            corr_matrix.iloc[i, j] = 1.0
        else:
            valid = df[[cat_cols[i], cat_cols[j]]].dropna()
            ct = pd.crosstab(valid[cat_cols[i]], valid[cat_cols[j]])
            cv = cramers_v(ct)
            corr_matrix.iloc[i, j] = cv
            corr_matrix.iloc[j, i] = cv

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(corr_matrix.astype(float), annot=True, fmt='.3f', cmap='YlOrRd', 
            vmin=0, vmax=0.6, ax=ax)
ax.set_title('Inter-Feature Association (Cramer\'s V)')
plt.tight_layout()
plt.savefig('./output_v8/feature_selection_multicollinearity.png', dpi=150, bbox_inches='tight')
plt.show()

print('\nHigh inter-feature associations (V > 0.30):')
for i in range(n_cats):
    for j in range(i+1, n_cats):
        v = corr_matrix.iloc[i, j]
        if v > 0.30:
            print(f'  {cat_cols[i]} <-> {cat_cols[j]}: V = {v:.3f}')

## 7. Summary and Recommendations

In [ ]:
print('=' * 70)
print('FEATURE SELECTION SUMMARY')
print('=' * 70)
print(f'\nDataset: 2024-07JUL.CSV')
print(f'Population: Working-age (15+) with valid PUFNEWEMPSTAT')
print(f'N = {len(df):,}')
print(f'\nTarget distribution (unweighted):')
for cls, label in empstat_labels.items():
    n = (df['PUFNEWEMPSTAT'] == cls).sum()
    print(f'  {label}: {n:,} ({n/len(df)*100:.1f}%)')

print(f'\n--- Feature Rankings ---')
print(f'\nRanked by effect size with employment status:')
for i, (_, r) in enumerate(results_df.iterrows(), 1):
    strength = 'LARGE' if r['effect_size'] > 0.30 else 'MEDIUM' if r['effect_size'] > 0.10 else 'SMALL' if r['effect_size'] > 0.01 else 'NEGLIGIBLE'
    in_all = 'Yes' if r['variable'] in ['PUFC04_SEX', 'PUFC05_AGE', 'age_group', 'PUFC06_MSTAT', 'grade_level', 'rel_group'] else 'No'
    print(f'  {i}. {r["feature"]:<28s} effect={r["effect_size"]:.4f} ({strength:<10s}) in_all_105_files={in_all}')

print(f'\n--- Variables that need careful harmonization ---')
print('Based on effect size > 0.01 AND available in all 105 files:')
for _, r in results_df.iterrows():
    if r['effect_size'] > 0.01 and r['variable'] in ['PUFC04_SEX', 'PUFC05_AGE', 'age_group', 'PUFC06_MSTAT', 'grade_level', 'rel_group']:
        print(f'  - {r["feature"]} ({r["variable"]}): effect={r["effect_size"]:.4f}')

print(f'\n--- Data leakage variables (EXCLUDED) ---')
excluded = [
    'PUFC11_WORK (worked last week) - determines employment status',
    'PUFC12_JOB (had job/business) - determines employment status',
    'PUFC14_PROCC (occupation) - only for employed',
    'PUFC16_PKB (industry) - only for employed',
    'PUFC17-29 (hours, pay, etc.) - only for employed',
    'PUFC30-43 (job search, reasons) - only for unemployed/NILF',
]
for e in excluded:
    print(f'  - {e}')